In [ ]:
pip install praat-parselmouth


In [ ]:
pip install --upgrade praat-parselmouth

In [ ]:
import numpy as np
import os
import librosa
import parselmouth
import soundfile

In [ ]:
acafly_xc109605 = '../input/birdclef-2021/train_short_audio/acafly/XC109605.ogg'
acowoo_xc110258 = '../input/birdclef-2021/train_short_audio/acowoo/XC110258.ogg'
yacafly,sry = librosa.load(acafly_xc109605)
acowoo,sro = librosa.load(acowoo_xc110258)
print("yacafly[:30]:{}\nyacfly.sr:{}".format(yacafly[:30],sry))
print("acowoo[:30]:{}\nyacfly.sr:{}".format(acowoo[:30],sro))

In [ ]:
def normalize(data):
    max = np.max(data)
    min = np.min(data)
    return (data-min)/(max-min)

In [ ]:
def get_5s_features(wavdata, sr):
    iters = int(np.ceil(len(wavdata) / (sr*5)))
    
    for i in range(iters):
        datas = wavdata[i * sr*5 : (i+1) * sr*5]
        print(len(datas))
        if not os.path.exists('../output/tmp/'):
             os.makedirs('../output/tmp/')
        soundfile.write('../output/tmp/tmp.wav', datas, sr)
        sound = parselmouth.Sound('../output/tmp/tmp.wav')
        frame = librosa.util.frame(datas, frame_length=1024, hop_length=512)
        energy = np.sum(frame**2, axis=0)
        n_energy = normalize(energy)
        mean_energy = np.mean(n_energy)
        std_energy = np.std(n_energy)
        
        zcr = librosa.feature.zero_crossing_rate(datas, frame_length=1024, hop_length=512, center=False)
        n_zcr = normalize(zcr)
        mean_zcr = np.mean(n_zcr)
        std_zcr = np.std(n_zcr)

        pitch = sound.to_pitch()  
        #print('pitch:{}'.format(pitch))
        q1 = parselmouth.praat.call(pitch, "Get quantile", 0.0, 0.0, 0.25, "Hertz")

        q2 = parselmouth.praat.call(pitch, "Get quantile", 0.0, 0.0, 0.5, "Hertz")
        q3 = parselmouth.praat.call(pitch, "Get quantile", 0.0, 0.0, 0.75, "Hertz")
        
        pitch_max = parselmouth.praat.call(pitch, "Get maximum", 0, 0, "Hertz", "Parabolic")
        pitch_min = parselmouth.praat.call(pitch, "Get minimum", 0, 0, "Hertz", "Parabolic")
        pitch_sd = parselmouth.praat.call(pitch, "Get standard deviation", 0, 0, "Hertz")
        
        mean_pitch = parselmouth.praat.call(pitch, "Get mean", 0.0, 0.0, "Hertz")
        
        formant = sound.to_formant_burg()
        mean_formant = []
        for idx in range(1, 6):
            mean_formant_i = parselmouth.praat.call(formant, "Get mean", float(idx), 0.0, 0.0, "Hertz")
            mean_formant.append(str(float(mean_formant_i)/1000))
            
        print(str(i*5)+'-'+ str((i+1)*5) +'\t'+ str(mean_energy)+ '\t'+ str(std_energy)+ '\t'+ str(mean_zcr)+ '\t'+ str(std_zcr)+ '\t'+ 
                str(float(q1)/1000)+ '\t'+ str(float(q2)/1000)+ '\t'+ str(float(q3)/1000)+ '\t'+ str(float(pitch_max)/1000)+ '\t'+ 
                str(float(pitch_min)/1000)+ '\t'+ str(float(pitch_sd)/1000)+ '\t' + str(float(mean_pitch)/1000)+ '\t'+ '\t'.join(mean_formant)
             )
print("-"*100)
get_5s_features(yacafly,sry)
print("\n\n"+"-"*100)
get_5s_features(yacafly,sry)
print("-"*100)